In [8]:
import numpy as np
import pandas as pd
import os
import re
from datetime import date
import sys
import mariadb
from pandas.io import sql
import sqlalchemy

import warnings    # to avoid warning during executions
warnings.filterwarnings("ignore")

## PBR lookup data preprocessing & DB export

- Load the data 
- populate unit and species column with DB tables primary key
- Export data into the pbr_lookpup table

In [9]:
# set the file path
file_name = file_name = '..\..\dataExport\\tbl_pbr_lookup.xlsx'

In [10]:
# load the excel data sheet and display first 5 rows
pbr_lookup = pd.read_excel(file_name,sheet_name='Sheet1')
pbr_lookup.head(5)

,instance,unit,species,start_date,end_date,remark
0,1.0,FPC13,1.0,1.629468e+09,1.633939e+09,NaN
1,2.0,FPC13,1.0,1.634652e+09,1.644489e+09,NaN
2,3.0,FPC13,11.0,1.625227e+09,1.628158e+09,NaN
3,4.0,FPC14,2.0,1.614764e+09,1.618393e+09,NaN
4,5.0,FPC14,7.0,1.625486e+09,1.639564e+09,NaN


In [11]:
print(pbr_lookup.shape)
print(pbr_lookup.info())

(12, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instance    12 non-null     float64
 1   unit        12 non-null     object 
 2   species     12 non-null     float64
 3   start_date  12 non-null     float64
 4   end_date    12 non-null     float64
 5   remark      0 non-null      float64
dtypes: float64(5), object(1)
memory usage: 704.0+ bytes
None


In [6]:
# database connection definition
database_username = 'root'
database_password = 'password'
database_ip       = '127.0.0.1:3306'
database_name     = 'data_dashboard'
database_connection = sqlalchemy.create_engine('mariadb+mariadbconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [13]:
# fetch data from mariadb table tbl_unit
tbl_unit = pd.read_sql('SELECT unit_id, unit_name FROM tbl_unit', database_connection)
tbl_unit.head(5)

,unit_id,unit_name
0,31,FFP_AUTFU
1,19,FFP_FFP
2,3,FFU
3,14,FPC11
4,34,FPC11_AUT11


In [14]:
# replace unit names with primary key value
pbr_lookup['unit'] = pbr_lookup['unit'].map(tbl_unit.set_index('unit_name')['unit_id'])

In [15]:
pbr_lookup

,instance,unit,species,start_date,end_date,remark
0,1.0,8,1.0,1.629468e+09,1.633939e+09,NaN
1,2.0,8,1.0,1.634652e+09,1.644489e+09,NaN
2,3.0,8,11.0,1.625227e+09,1.628158e+09,NaN
3,4.0,2,2.0,1.614764e+09,1.618393e+09,NaN
4,5.0,2,7.0,1.625486e+09,1.639564e+09,NaN
5,6.0,2,11.0,1.618562e+09,1.625229e+09,NaN
6,7.0,6,2.0,1.634915e+09,1.646645e+09,NaN
7,8.0,6,9.0,1.623051e+09,1.626772e+09,NaN
8,9.0,4,3.0,1.620216e+09,1.634632e+09,NaN
9,10.0,7,7.0,1.616427e+09,1.626691e+09,NaN


In [18]:
try:
    pbr_lookup.to_sql(con=database_connection, name='tbl_pbr_lookup', if_exists='append', index=False)
except:
    print("Duplicate Entry")



Duplicate Entry


### Appendix

df['CODE'] = df.apply(lambda x: 0 if x['distance'] in a else x['CODE'], axis=1)